# Leafletfinder to Edge List

Use Leaflet Finder Code to create an graph based on Oliver's MD trajectory.

**Data:**

In [8]:
!ls /data/leafletfinder/large/

63342lip_576TMprotein_nowat_10us_1us_timestep_fixed.xtc
63342lip_576TMprotein_nowat_start.pdb
cg_membrane1.jpg
cg_membrane_outlier_micelle1.jpg
cg_membrane.pdb.bz2
find_leaflets.py
leafletformation.py
leaflets_12x12.txt
leaflets_63342lip_576TMprotein_nowat_10us_1us_timestep_fixed.txt
md_centered.xtc
md.pdb
md_prod_12x12_everymicroS_pbcmolcenter.xtc
md_prod_12x12_lastframe.pdb
README_2.txt
TRPV1_VS_self_assembly_lipidgroups.pdf
vesicle_1_5M_373_first.gro
vesicle_1_5M_373_last.gro
vesicle_1_5M_373_stride1000.xtc
vesicle_1_5M_373.tpr


In [3]:
import MDAnalysis
topology = "/data/leafletfinder/large/vesicle_1_5M_373.tpr"
trajectory = "/data/leafletfinder/large/vesicle_1_5M_373_stride1000.xtc"
u = MDAnalysis.Universe(topology, trajectory)

In [5]:
import MDAnalysis.analysis.leaflet
L = MDAnalysis.analysis.leaflet.LeafletFinder(u, "name P*", pbc=True)

MemoryError: 